# Bounty Hunting Project Notebook

David Feng and Karan Sampath

## Set Up

First, we load in some helper files.

In [ ]:
import sys
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
sys.path.append('dontlook')
import pandas as pd
from dontlook import bountyHuntData
from sklearn import metrics
from dontlook import bountyHuntWrapper
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

Next, we load in the data. You should use `train_x` and `train_y` to train your models. The second set of data (`validation_x` and `validation_y`) is for testing your models, to ensure that you aren't overfitting. It is also what will be passed to the updater in order to determine if a proposed update should be accepted and if repairs are needed. Since you have access to this data, you could overfit to it and get a bunch of updates accepted. However, a) we'll be able to tell you did this and b) your updates will fail on the holdout set that only we have access to, so doing this is not in your best interest.

In [ ]:
[train_x, train_y, validation_x, validation_y] = bountyHuntData.get_data()

The model that you'll be building off of is a decision stump, i.e. a very stupid decision list with only one node. **Warning: do not rerun the next code block unless you want to completely restart building your PDL, as it will re-initialize it to just the decision stump!**

In [ ]:
initial_model = DecisionTreeClassifier(max_depth = 10, random_state=0)
initial_model.fit(train_x, train_y)
f = bountyHuntWrapper.build_initial_pdl(initial_model, train_x, train_y, validation_x, validation_y)

# Bounty Hunting

Here's where the bulk of the work you'll be doing will live. Your job is to generate groups g such that there is some h that does better than the current model f on that group. Here, we generate an example group function, which identifies African American individuals.

In [ ]:
def g1(x):
    """
    Given an x, g should return either 0 or 1.
    :param x: input vector
    :return: 0 or 1; 1 if x belongs to group, 0 otherwise
    """

    # here is how to make a function g that returns 1 for all African American individuals
    if x['RAC1P'] == 2:
        return 1
    else:
        return 0

You might also imaging making a group function that tries to learn what regions the current algorithm performs poorly on in an adaptive way, instead of just guessing ad-hoc that it will do poorly on a particular subgroup. In order to generate such a g, you will need to generate a constructor that takes as input a current model and the training data, and outputs a function g. A template for doing this is provided below. The example version returns a very silly function g which looks at the predictions the current model makes, and returns a group function where the group it has learned is all the points that the PDL labels as a 1. It completely ignores the true labels (train_y), so is probably not a very good group function.

In [ ]:
import numpy as np


def g_(f,train_x, train_y):
    # f is the current PDL
    preds = train_x.apply(f.predict, axis=1)
    xs = train_x[preds == 1]
    ys = train_y[preds == 1]
    dt = DecisionTreeClassifier(max_depth = 1, random_state=0)
    dt.fit(xs, ys)
    def g(x):
        # g should take as input a SINGLE x and return 0 or 1 for it.
        # if we call dt.predict on x it will break because the dimensions of x are wrong, so we have to reshape it and reshape the output.
        # this is not particularly efficient, so if you have better ways of doing this go for it. :)
        y = dt.predict(np.array(x).reshape(1,-1))
        return y[0]
    return g

# if you wanted to build a particular g using the above, you could use the following line.
g = g_(f, train_x, train_y)

In the following cell(s), generate group functions that you think will make improvements, and then try to run the updates as explained in the subsequent section. In the final version of your code that you turn in, the groups that you generated and their corresponding models h, and the order in which you did updates, should be obvious and re-generating your final PDL should be completely reproducible just by running the code blocks in this notebook.

# College Age Model (Ages 18-22)

Through the data exploration, we realized that there were certain groups that did not perform well in our initial model. The worst performers seemed to be college-age kids (18-22) and retirement age folk (60-70). This makes sense, as some people don't go to college and start working while others are full-time students. Also, some people may still be working while they are of "retirement age" while others may have already settled down. 

In [ ]:
# Add your code defining groups (and possibly hs) here.
def group_age(lower, upper, x):
  if x['AGEP'] <= upper and x['AGEP'] >= lower:
    return True
  else:
    return False

def group_age_college(x):
  if x['AGEP'] <= 22 and x['AGEP'] >= 18:
    return True
  else:
    return False

# first train a model on people ages between 18 and 22, which are college age kids
# getting the right training set
college_age_x = train_x.apply(lambda x: group_age(18, 22, x), axis=1)
college_train_y = train_y[college_age_x]
college_train_x = train_x[college_age_x]

# check on validation set
college_v = validation_x.apply(lambda x: group_age(18, 22, x), axis=1)
college_v_x = validation_x[college_v]
college_v_y = validation_y[college_v]

We are going to specify the range of hyperparameters to better tune the Random Forest Model we are using.

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(7, 18, num = 10)]
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [1, 2, 4, 6]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


Next we run a RandomizedSearch with Cross Validation to find the best hyperparameters out of these combinations. That is the model we will use to update the PDL.

In [ ]:
college_clf = RandomForestClassifier()
college_clf_rand = RandomizedSearchCV(estimator = college_clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
college_clf_rand.fit(college_train_x, college_train_y)
print(college_clf_rand.best_params_)


In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [9, 10, 11],
    'max_features': [2, 3],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [500, 700, 900]
}

grid_search_college = GridSearchCV(estimator = college_clf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search_college.fit(college_train_x, college_train_y)
print(grid_search_college.best_params_)

In [ ]:
college_clf = RandomForestClassifier(n_estimators=700, max_depth=9, bootstrap=True, max_features=3, min_samples_leaf=4, min_samples_split=12, random_state=69)
college_clf.fit(college_train_x, college_train_y)
college_train_pred = college_clf.predict(college_train_x)
college_val_pred = college_clf.predict(college_v_x)
print("Training Error")
print(metrics.zero_one_loss(college_train_y, college_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(college_v_y, college_val_pred))

We can also try another model: the support vector machine (SVM)!

In [ ]:
college_svm = svm.SVC()
param_grid_svm = {
  'kernel': ['linear', 'polynomial', 'RBF']
}

grid_search_college_svm = GridSearchCV(estimator = college_svm, param_grid = param_grid_svm, cv = 5, n_jobs = -1, verbose = 2)
grid_search_college_svm.fit(college_train_x, college_train_y)
print(grid_search_college_svm.best_params_)

In [ ]:
college_svm = svm.SVC(kernel='linear')
college_svm.fit(college_train_x, college_train_y)
college_svm_train_pred = college_svm.predict(college_train_x)
college_svm_val_pred = college_svm.predict(college_v_x)
print("Training Error")
print(metrics.zero_one_loss(college_train_y, college_svm_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(college_v_y, college_svm_val_pred))

It seems as if the Random Forest Model performed better! Let's use that one to update our PDL.

In [ ]:
def simple_updater(g,group_name = "g"):
    # if you want to change how h is trained, you can edit the below line.
    h = bountyHuntWrapper.build_model(train_x, train_y, g, dt_depth=10)
    # do not change anything beyond this point.
    if bountyHuntWrapper.run_checks(f, validation_x, validation_y, g, h, train_x=train_x, train_y=train_y):
        print("Running Update")
        bountyHuntWrapper.run_updates(f, g, h, train_x, train_y, validation_x, validation_y, group_name=group_name)

def updater(g, h, group_name="g"):
    # do not alter this code
    if bountyHuntWrapper.run_checks(f, validation_x, validation_y, g, h, train_x=train_x, train_y=train_y):
        print("Running Update")
        bountyHuntWrapper.run_updates(f, g, h, train_x, train_y, validation_x, validation_y, group_name=group_name)

In [ ]:
updater(group_age_college, college_clf.predict, group_name="college age")

# Just 20 year olds (45% error currently... Yikes!)

In [ ]:
def group_age_20(x):
  if x['AGEP'] == 20:
    return True
  else:
    return False

# first train a model on people ages between 18 and 22, which are college age kids
# getting the right training set
twenty_tf = train_x.apply(lambda x: group_age_20(x), axis=1)
twenty_train_y = train_y[twenty_tf]
twenty_train_x = train_x[twenty_tf]

# check on validation set
twenty_v = validation_x.apply(lambda x: group_age_20(x), axis=1)
twenty_v_x = validation_x[twenty_v]
twenty_v_y = validation_y[twenty_v]

In [ ]:
twenty_clf = RandomForestClassifier()
twenty_clf_rand = RandomizedSearchCV(estimator = twenty_clf, param_distributions = random_grid, n_iter = 70, cv = 3, verbose=2, random_state=42, n_jobs = -1)
twenty_clf_rand.fit(twenty_train_x, twenty_train_y)
print(twenty_clf_rand.best_params_)

In [ ]:
param_grid_twenty = {
    'bootstrap': [False],
    'max_depth': [9, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [2, 3],
    'min_samples_split': [5, 7],
    'n_estimators': [500, 700]
}

grid_search_twenty = GridSearchCV(estimator = twenty_clf, param_grid = param_grid_twenty, cv = 3, n_jobs = -1, verbose = 2)
grid_search_twenty.fit(twenty_train_x, twenty_train_y)
print(grid_search_twenty.best_params_)

In [ ]:
twenty_clf = RandomForestClassifier(n_estimators=700, max_depth=9, bootstrap=False, max_features=3, min_samples_leaf=2, min_samples_split=5, random_state=69)
twenty_clf.fit(twenty_train_x, twenty_train_y)
twenty_train_pred = twenty_clf.predict(twenty_train_x)
twenty_val_pred = twenty_clf.predict(twenty_v_x)
print("Training Error")
print(metrics.zero_one_loss(twenty_train_y, twenty_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(twenty_v_y, twenty_val_pred))

# Age 64

In [ ]:
def group_age_64(x):
  if x['AGEP'] == 64:
    return True
  else:
    return False

# first train a model on people ages between 18 and 22, which are college age kids
# getting the right training set
sixty_four_tf = train_x.apply(lambda x: group_age_64(x), axis=1)
sixty_four_train_y = train_y[sixty_four_tf]
sixty_four_train_x = train_x[sixty_four_tf]

# check on validation set
sixty_four_v = validation_x.apply(lambda x: group_age_64(x), axis=1)
sixty_four_v_x = validation_x[sixty_four_v]
sixty_four_v_y = validation_y[sixty_four_v]

In [ ]:
sixty_four_clf = RandomForestClassifier()
sixty_four_clf_rand = RandomizedSearchCV(estimator = sixty_four_clf, param_distributions = random_grid, n_iter = 64, cv = 3, verbose=2, random_state=42, n_jobs = -1)
sixty_four_clf_rand.fit(sixty_four_train_x, sixty_four_train_y)
print(sixty_four_clf_rand.best_params_)

In [ ]:
param_grid_sixty_four = {
    'bootstrap': [True],
    'max_depth': [9, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [6, 7],
    'n_estimators': [800, 1000]
}

grid_search_sixty_four = GridSearchCV(estimator = sixty_four_clf, param_grid = param_grid_sixty_four, cv = 3, n_jobs = -1, verbose = 2)
grid_search_sixty_four.fit(sixty_four_train_x, sixty_four_train_y)
print(grid_search_sixty_four.best_params_)

In [ ]:
sixty_four_clf = RandomForestClassifier(n_estimators=800, max_depth=10, bootstrap=True, max_features=3, min_samples_leaf=1, min_samples_split=6, random_state=69)
sixty_four_clf.fit(sixty_four_train_x, sixty_four_train_y)
sixty_four_train_pred = sixty_four_clf.predict(sixty_four_train_x)
sixty_four_val_pred = sixty_four_clf.predict(sixty_four_v_x)
print("Training Error")
print(metrics.zero_one_loss(sixty_four_train_y, sixty_four_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sixty_four_v_y, sixty_four_val_pred))

# Age 19
Ahh... young people! To be or not to be (employed)!

In [ ]:
def group_age_19(x):
  if x['AGEP'] == 19:
    return True
  else:
    return False

# first train a model on people ages between 18 and 22, which are college age kids
# getting the right training set
nineteen_tf = train_x.apply(lambda x: group_age_19(x), axis=1)
nineteen_train_y = train_y[nineteen_tf]
nineteen_train_x = train_x[nineteen_tf]

# check on validation set
nineteen_v = validation_x.apply(lambda x: group_age_19(x), axis=1)
nineteen_v_x = validation_x[nineteen_v]
nineteen_v_y = validation_y[nineteen_v]

In [ ]:
nineteen_clf = RandomForestClassifier()
nineteen_clf_rand = RandomizedSearchCV(estimator = nineteen_clf, param_distributions = random_grid, n_iter = 64, cv = 3, verbose=2, random_state=42, n_jobs = -1)
nineteen_clf_rand.fit(nineteen_train_x, nineteen_train_y)
print(nineteen_clf_rand.best_params_)

In [ ]:
param_grid_nineteen = {
    'bootstrap': [True],
    'max_depth': [10, 11, 12],
    'max_features': [2, 3],
    'min_samples_leaf': [6, 7],
    'min_samples_split': [2, 3],
    'n_estimators': [100, 200, 300]
}

grid_search_nineteen = GridSearchCV(estimator = nineteen_clf, param_grid = param_grid_nineteen, cv = 3, n_jobs = -1, verbose = 2)
grid_search_nineteen.fit(nineteen_train_x, nineteen_train_y)
print(grid_search_nineteen.best_params_)

In [ ]:
nineteen_clf = RandomForestClassifier(n_estimators=300, max_depth=11, bootstrap=True, max_features=2, min_samples_leaf=6, min_samples_split=2, random_state=69)
nineteen_clf.fit(nineteen_train_x, nineteen_train_y)
nineteen_train_pred = nineteen_clf.predict(nineteen_train_x)
nineteen_val_pred = nineteen_clf.predict(nineteen_v_x)
print("Training Error")
print(metrics.zero_one_loss(nineteen_train_y, nineteen_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(nineteen_v_y, nineteen_val_pred))

# Age 21

In [ ]:
def group_age_21(x):
  if x['AGEP'] == 21:
    return True
  else:
    return False

# getting the right training set
twenty_one_tf = train_x.apply(lambda x: group_age_21(x), axis=1)
twenty_one_train_y = train_y[twenty_one_tf]
twenty_one_train_x = train_x[twenty_one_tf]

# check on validation set
twenty_one_v = validation_x.apply(lambda x: group_age_21(x), axis=1)
twenty_one_v_x = validation_x[twenty_one_v]
twenty_one_v_y = validation_y[twenty_one_v]

In [ ]:
twenty_one_clf = AdaBoostClassifier(n_estimators=200, random_state=0)
twenty_one_clf.fit(twenty_one_train_x, twenty_one_train_y)
twenty_one_train_pred = twenty_one_clf.predict(twenty_one_train_x)
twenty_one_val_pred = twenty_one_clf.predict(twenty_one_v_x)
print("Training Error")
print(metrics.zero_one_loss(twenty_one_train_y, twenty_one_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(twenty_one_v_y, twenty_one_val_pred))

In [ ]:
param_grid_twenty_one = {
    'bootstrap': [True],
    'max_depth': [9, 10, 11],
    'max_features': [2, 3],
    'min_samples_leaf': [6, 7],
    'min_samples_split': [6, 7],
    'n_estimators': [600, 700]
}

grid_search_twenty_one = GridSearchCV(estimator = twenty_one_clf, param_grid = param_grid_twenty_one, cv = 3, n_jobs = -1, verbose = 2)
grid_search_twenty_one.fit(twenty_one_train_x, twenty_one_train_y)
print(grid_search_twenty_one.best_params_)

In [ ]:
twenty_one_clf = RandomForestClassifier(n_estimators=600, max_depth=11, bootstrap=True, max_features=4, min_samples_leaf=2, min_samples_split=4, random_state=69)
twenty_one_clf.fit(twenty_one_train_x, twenty_one_train_y)
twenty_one_train_pred = twenty_one_clf.predict(twenty_one_train_x)
twenty_one_val_pred = twenty_one_clf.predict(twenty_one_v_x)
print("Training Error")
print(metrics.zero_one_loss(twenty_one_train_y, twenty_one_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(twenty_one_v_y, twenty_one_val_pred))

# Age 63

In [ ]:
def group_age_63(x):
  if x['AGEP'] == 63:
    return True
  else:
    return False

# getting the right training set
sixty_three_tf = train_x.apply(lambda x: group_age_63(x), axis=1)
sixty_three_train_y = train_y[sixty_three_tf]
sixty_three_train_x = train_x[sixty_three_tf]

# check on validation set
sixty_three_v = validation_x.apply(lambda x: group_age_63(x), axis=1)
sixty_three_v_x = validation_x[sixty_three_v]
sixty_three_v_y = validation_y[sixty_three_v]

In [ ]:
sixty_three_clf = RandomForestClassifier()
sixty_three_clf_rand = RandomizedSearchCV(estimator = sixty_three_clf, param_distributions = random_grid, n_iter = 70, cv = 3, verbose=2, random_state=42, n_jobs = -1)
sixty_three_clf_rand.fit(sixty_three_train_x, sixty_three_train_y)
print(sixty_three_clf_rand.best_params_)

In [ ]:
param_grid_sixty_three = {
    'bootstrap': [True],
    'max_depth': [7, 9, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [6, 7],
    'n_estimators': [400, 500]
}

grid_search_sixty_three = GridSearchCV(estimator = sixty_three_clf, param_grid = param_grid_sixty_three, cv = 3, n_jobs = -1, verbose = 2)
grid_search_sixty_three.fit(sixty_three_train_x, sixty_three_train_y)
print(grid_search_sixty_three.best_params_)

In [ ]:
sixty_three_clf = RandomForestClassifier(bootstrap=True, max_depth=10, max_features=2, min_samples_leaf=2, min_samples_split=7, n_estimators=500)
sixty_three_clf.fit(sixty_three_train_x, sixty_three_train_y)
sixty_three_train_pred = sixty_three_clf.predict(sixty_three_train_x)
sixty_three_val_pred = sixty_three_clf.predict(sixty_three_v_x)

print("Training Error")
print(metrics.zero_one_loss(sixty_three_train_y, sixty_three_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sixty_three_v_y, sixty_three_val_pred))

# Age 65

In [ ]:
def group_age_65(x):
  if x['AGEP'] == 65:
    return True
  else:
    return False

def group_age_retirement(x):
  if x['AGEP'] <= 70 and x['AGEP'] >= 60:
    return True
  else:
    return False

# get the correct training set for retirement
retirement_age = train_x.apply(lambda x: group_age_retirement(x), axis=1)
retirement_train_y = train_y[retirement_age]
retirement_train_x = train_x[retirement_age]

# check on validation set
retirement_age_v = validation_x.apply(lambda x: group_age_retirement(x), axis=1)
retirement_v_x = validation_x[retirement_age_v]
retirement_v_y = validation_y[retirement_age_v]

# getting the right training set for 65
sixty_five_tf = train_x.apply(lambda x: group_age_65(x), axis=1)
sixty_five_train_y = train_y[sixty_five_tf]
sixty_five_train_x = train_x[sixty_five_tf]

# check on validation set
sixty_five_v = validation_x.apply(lambda x: group_age_65(x), axis=1)
sixty_five_v_x = validation_x[sixty_five_v]
sixty_five_v_y = validation_y[sixty_five_v]

sixty_five_clf = RandomForestClassifier(bootstrap=True, max_depth=12, max_features=3, min_samples_leaf=4, min_samples_split=7, n_estimators=500)
sixty_five_clf.fit(retirement_train_x, retirement_train_y)

In [ ]:
sixty_five_clf = RandomForestClassifier()
sixty_five_clf_rand = RandomizedSearchCV(estimator = sixty_five_clf, param_distributions = random_grid, n_iter = 70, cv = 3, verbose=2, random_state=42, n_jobs = -1)
sixty_five_clf_rand.fit(sixty_five_train_x, sixty_five_train_y)
print(sixty_five_clf_rand.best_params_)

In [ ]:
param_grid_sixty_five = {
    'bootstrap': [True],
    'max_depth': [10, 11, 12],
    'max_features': [2, 3],
    'min_samples_leaf': [6, 7],
    'min_samples_split': [2, 3],
    'n_estimators': [100, 200]
}

sixty_five_clf = RandomForestClassifier()
grid_search_sixty_five = GridSearchCV(estimator = sixty_five_clf, param_grid = param_grid_sixty_five, cv = 3, n_jobs = -1, verbose = 2)
grid_search_sixty_five.fit(sixty_five_train_x, sixty_five_train_y)
print(grid_search_sixty_five.best_params_)

In [ ]:
sixty_five_clf = RandomForestClassifier(bootstrap=True, max_depth=12, max_features=3, min_samples_leaf=4, min_samples_split=7, n_estimators=500)
sixty_five_clf.fit(retirement_train_x, retirement_train_y)
sixty_five_train_pred = sixty_five_clf.predict(sixty_five_train_x)
sixty_five_val_pred = sixty_five_clf.predict(sixty_five_v_x)

print("Training Error")
print(metrics.zero_one_loss(sixty_five_train_y, sixty_five_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sixty_five_v_y, sixty_five_val_pred))


# Age 67

In [ ]:
def group_age_67(x):
  if x['AGEP'] == 67:
    return True
  else:
    return False

# getting the right training set
sixty_seven_tf = train_x.apply(lambda x: group_age_67(x), axis=1)
sixty_seven_train_y = train_y[sixty_seven_tf]
sixty_seven_train_x = train_x[sixty_seven_tf]

# check on validation set
sixty_seven_v = validation_x.apply(lambda x: group_age_67(x), axis=1)
sixty_seven_v_x = validation_x[sixty_seven_v]
sixty_seven_v_y = validation_y[sixty_seven_v]

sixty_seven_clf = RandomForestClassifier(bootstrap=True, max_depth=12, max_features=3, min_samples_leaf=4, min_samples_split=7, n_estimators=500)
sixty_seven_clf.fit(sixty_seven_train_x, sixty_seven_train_y)
sixty_seven_train_pred = sixty_seven_clf.predict(sixty_seven_train_x)
sixty_seven_val_pred = sixty_seven_clf.predict(sixty_seven_v_x)

print("Training Error")
print(metrics.zero_one_loss(sixty_seven_train_y, sixty_seven_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sixty_seven_v_y, sixty_seven_val_pred))


# Age 61

In [ ]:
def group_age_61(x):
  if x['AGEP'] == 61:
    return True
  else:
    return False

# getting the right training set
sixty_one_tf = train_x.apply(lambda x: group_age_61(x), axis=1)
sixty_one_train_y = train_y[sixty_one_tf]
sixty_one_train_x = train_x[sixty_one_tf]

# check on validation set
sixty_one_v = validation_x.apply(lambda x: group_age_61(x), axis=1)
sixty_one_v_x = validation_x[sixty_one_v]
sixty_one_v_y = validation_y[sixty_one_v]

sixty_one_clf = RandomForestClassifier(bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
sixty_one_clf.fit(retirement_train_x, retirement_train_y)
sixty_one_train_pred = sixty_one_clf.predict(sixty_one_train_x)
sixty_one_val_pred = sixty_one_clf.predict(sixty_one_v_x)

print("Training Error")
print(metrics.zero_one_loss(sixty_one_train_y, sixty_one_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sixty_one_v_y, sixty_one_val_pred))


# Age 60

In [ ]:
def group_age_60(x):
  if x['AGEP'] == 60:
    return True
  else:
    return False

def group_age_57_62(x):
  if x['AGEP'] >= 57 and x['AGEP'] <= 62:
    return True
  else:
    return False

# getting the right training set
sixty_tf = train_x.apply(lambda x: group_age_60(x), axis=1)
sixty_train_y = train_y[sixty_tf]
sixty_train_x = train_x[sixty_tf]

# getting the right training set
range_sixty_tf = train_x.apply(lambda x: group_age_57_62(x), axis=1)
range_sixty_train_y = train_y[range_sixty_tf]
range_sixty_train_x = train_x[range_sixty_tf]

# check on validation set
sixty_v = validation_x.apply(lambda x: group_age_60(x), axis=1)
sixty_v_x = validation_x[sixty_v]
sixty_v_y = validation_y[sixty_v]

sixty_clf = RandomForestClassifier(bootstrap=True, max_depth=8, max_features=3, min_samples_leaf=3, min_samples_split=6, n_estimators=500)
sixty_clf.fit(range_sixty_train_x, range_sixty_train_y)
sixty_train_pred = sixty_clf.predict(sixty_train_x)
sixty_val_pred = sixty_clf.predict(sixty_v_x)

print("Training Error")
print(metrics.zero_one_loss(sixty_train_y, sixty_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sixty_v_y, sixty_val_pred))


# Age 23

In [ ]:
def group_age_23(x):
  if x['AGEP'] == 23:
    return True
  else:
    return False

def group_age_22_25(x):
  if x['AGEP'] >= 22 and x['AGEP'] <= 25:
    return True
  else:
    return False

# getting the right training set
twenty_three_tf = train_x.apply(lambda x: group_age_23(x), axis=1)
twenty_three_train_y = train_y[twenty_three_tf]
twenty_three_train_x = train_x[twenty_three_tf]

# getting the right training set
range_twenty_three_tf = train_x.apply(lambda x: group_age_22_25(x), axis=1)
range_twenty_three_train_y = train_y[range_twenty_three_tf]
range_twenty_three_train_x = train_x[range_twenty_three_tf]

# check on validation set
twenty_three_v = validation_x.apply(lambda x: group_age_23(x), axis=1)
twenty_three_v_x = validation_x[twenty_three_v]
twenty_three_v_y = validation_y[twenty_three_v]

twenty_three_clf = RandomForestClassifier(bootstrap=True, max_depth=8, max_features=3, min_samples_leaf=3, min_samples_split=6, n_estimators=500)
twenty_three_clf.fit(twenty_three_train_x, twenty_three_train_y)
twenty_three_train_pred = twenty_three_clf.predict(twenty_three_train_x)
twenty_three_val_pred = twenty_three_clf.predict(twenty_three_v_x)

print("Training Error")
print(metrics.zero_one_loss(twenty_three_train_y, twenty_three_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(twenty_three_v_y, twenty_three_val_pred))


# SEX = 2

In [ ]:
def group_sex_2(x):
  if x['SEX'] == 2:
    return True
  else:
    return False

# getting the right training set
sex_2_tf = train_x.apply(lambda x: group_sex_2(x), axis=1)
sex_2_train_y = train_y[sex_2_tf]
sex_2_train_x = train_x[sex_2_tf]

# check on validation set
sex_2_v = validation_x.apply(lambda x: group_sex_2(x), axis=1)
sex_2_v_x = validation_x[sex_2_v]
sex_2_v_y = validation_y[sex_2_v]

sex_2_clf = RandomForestClassifier(bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
sex_2_clf.fit(sex_2_train_x, sex_2_train_y)
sex_2_train_pred = sex_2_clf.predict(sex_2_train_x)
sex_2_val_pred = sex_2_clf.predict(sex_2_v_x)

print("Training Error")
print(metrics.zero_one_loss(sex_2_train_y, sex_2_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sex_2_v_y, sex_2_val_pred))

# CIT=4

In [ ]:
def group_cit_4(x):
  if x['CIT'] == 4:
    return True
  else:
    return False

# getting the right training set
cit_4_tf = train_x.apply(lambda x: group_cit_4(x), axis=1)
cit_4_train_y = train_y[cit_4_tf]
cit_4_train_x = train_x[cit_4_tf]

# check on validation set
cit_4_v = validation_x.apply(lambda x: group_cit_4(x), axis=1)
cit_4_v_x = validation_x[cit_4_v]
cit_4_v_y = validation_y[cit_4_v]

cit_4_clf = RandomForestClassifier(bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
cit_4_clf.fit(cit_4_train_x, cit_4_train_y)
cit_4_train_pred = cit_4_clf.predict(cit_4_train_x)
cit_4_val_pred = cit_4_clf.predict(cit_4_v_x)

print("Training Error")
print(metrics.zero_one_loss(cit_4_train_y, cit_4_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(cit_4_v_y, cit_4_val_pred))

# MAR=3

In [ ]:
def group_mar_3(x):
  if x['MAR'] == 3:
    return True
  else:
    return False

# getting the right training set
mar_3_tf = train_x.apply(lambda x: group_mar_3(x), axis=1)
mar_3_train_y = train_y[mar_3_tf]
mar_3_train_x = train_x[mar_3_tf]

# check on validation set
mar_3_v = validation_x.apply(lambda x: group_mar_3(x), axis=1)
mar_3_v_x = validation_x[mar_3_v]
mar_3_v_y = validation_y[mar_3_v]

mar_3_clf = RandomForestClassifier(bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
mar_3_clf.fit(mar_3_train_x, mar_3_train_y)
mar_3_train_pred = mar_3_clf.predict(mar_3_train_x)
mar_3_val_pred = mar_3_clf.predict(mar_3_v_x)

print("Training Error")
print(metrics.zero_one_loss(mar_3_train_y, mar_3_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(mar_3_v_y, mar_3_val_pred))

# RAC1P = 6

In [ ]:
def group_rac1p_6(x):
  if x['RAC1P'] == 6:
    return True
  else:
    return False

# getting the right training set
rac1p_6_tf = train_x.apply(lambda x: group_rac1p_6(x), axis=1)
rac1p_6_train_y = train_y[rac1p_6_tf]
rac1p_6_train_x = train_x[rac1p_6_tf]

# check on validation set
rac1p_6_v = validation_x.apply(lambda x: group_rac1p_6(x), axis=1)
rac1p_6_v_x = validation_x[rac1p_6_v]
rac1p_6_v_y = validation_y[rac1p_6_v]

rac1p_6_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
rac1p_6_clf.fit(rac1p_6_train_x, rac1p_6_train_y)
rac1p_6_train_pred = rac1p_6_clf.predict(rac1p_6_train_x)
rac1p_6_val_pred = rac1p_6_clf.predict(rac1p_6_v_x)

print("Training Error")
print(metrics.zero_one_loss(rac1p_6_train_y, rac1p_6_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(rac1p_6_v_y, rac1p_6_val_pred))

# MIL = 4 (Put at end because it's a large group)

In [ ]:
def group_mil_4(x):
  if x['MIL'] == 4:
    return True
  else:
    return False

# getting the right training set
mil_4_tf = train_x.apply(lambda x: group_mil_4(x), axis=1)
mil_4_train_y = train_y[mil_4_tf]
mil_4_train_x = train_x[mil_4_tf]

# check on validation set
mil_4_v = validation_x.apply(lambda x: group_mil_4(x), axis=1)
mil_4_v_x = validation_x[mil_4_v]
mil_4_v_y = validation_y[mil_4_v]

mil_4_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
mil_4_clf.fit(mil_4_train_x, mil_4_train_y)
mil_4_train_pred = mil_4_clf.predict(mil_4_train_x)
mil_4_val_pred = mil_4_clf.predict(mil_4_v_x)

print("Training Error")
print(metrics.zero_one_loss(mil_4_train_y, mil_4_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(mil_4_v_y, mil_4_val_pred))

# Nativity=2

In [ ]:
def group_nativity_2(x):
  if x['NATIVITY'] == 2:
    return True
  else:
    return False

# getting the right training set
nativity_2_tf = train_x.apply(lambda x: group_nativity_2(x), axis=1)
nativity_2_train_y = train_y[nativity_2_tf]
nativity_2_train_x = train_x[nativity_2_tf]

# check on validation set
nativity_2_v = validation_x.apply(lambda x: group_nativity_2(x), axis=1)
nativity_2_v_x = validation_x[nativity_2_v]
nativity_2_v_y = validation_y[nativity_2_v]

nativity_2_clf = RandomForestClassifier(bootstrap=True, max_depth=8, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
nativity_2_clf.fit(nativity_2_train_x, nativity_2_train_y)
nativity_2_train_pred = nativity_2_clf.predict(nativity_2_train_x)
nativity_2_val_pred = nativity_2_clf.predict(nativity_2_v_x)

print("Training Error")
print(metrics.zero_one_loss(nativity_2_train_y, nativity_2_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(nativity_2_v_y, nativity_2_val_pred))

# SCHL=18

In [ ]:
def group_schl_18(x):
  if x['SCHL'] == 18:
    return True
  else:
    return False

# getting the right training set
schl_18_tf = train_x.apply(lambda x: group_schl_18(x), axis=1)
schl_18_train_y = train_y[schl_18_tf]
schl_18_train_x = train_x[schl_18_tf]

# check on validation set
schl_18_v = validation_x.apply(lambda x: group_schl_18(x), axis=1)
schl_18_v_x = validation_x[schl_18_v]
schl_18_v_y = validation_y[schl_18_v]

schl_18_clf = RandomForestClassifier(bootstrap=True, max_depth=8, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
schl_18_clf.fit(schl_18_train_x, schl_18_train_y)
schl_18_train_pred = schl_18_clf.predict(schl_18_train_x)
schl_18_val_pred = schl_18_clf.predict(schl_18_v_x)

print("Training Error")
print(metrics.zero_one_loss(schl_18_train_y, schl_18_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(schl_18_v_y, schl_18_val_pred))

# SCHL=16

In [ ]:
def group_schl_16(x):
  if x['SCHL'] == 16:
    return True
  else:
    return False

# getting the right training set
schl_16_tf = train_x.apply(lambda x: group_schl_16(x), axis=1)
schl_16_train_y = train_y[schl_16_tf]
schl_16_train_x = train_x[schl_16_tf]

# check on validation set
schl_16_v = validation_x.apply(lambda x: group_schl_16(x), axis=1)
schl_16_v_x = validation_x[schl_16_v]
schl_16_v_y = validation_y[schl_16_v]

schl_16_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
schl_16_clf.fit(schl_16_train_x, schl_16_train_y)
schl_16_train_pred = schl_16_clf.predict(schl_16_train_x)
schl_16_val_pred = schl_16_clf.predict(schl_16_v_x)

print("Training Error")
print(metrics.zero_one_loss(schl_16_train_y, schl_16_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(schl_16_v_y, schl_16_val_pred))

# RELP=1

In [ ]:
def group_relp_1(x):
  if x['RELP'] == 1:
    return True
  else:
    return False

# getting the right training set
relp_1_tf = train_x.apply(lambda x: group_relp_1(x), axis=1)
relp_1_train_y = train_y[relp_1_tf]
relp_1_train_x = train_x[relp_1_tf]

# check on validation set
relp_1_v = validation_x.apply(lambda x: group_relp_1(x), axis=1)
relp_1_v_x = validation_x[relp_1_v]
relp_1_v_y = validation_y[relp_1_v]

relp_1_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
relp_1_clf.fit(relp_1_train_x, relp_1_train_y)
relp_1_train_pred = relp_1_clf.predict(relp_1_train_x)
relp_1_val_pred = relp_1_clf.predict(relp_1_v_x)

print("Training Error")
print(metrics.zero_one_loss(relp_1_train_y, relp_1_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(relp_1_v_y, relp_1_val_pred))

# RELP=17

In [ ]:
def group_relp_17(x):
  if x['RELP'] == 17:
    return True
  else:
    return False

# getting the right training set
relp_17_tf = train_x.apply(lambda x: group_relp_17(x), axis=1)
relp_17_train_y = train_y[relp_17_tf]
relp_17_train_x = train_x[relp_17_tf]

# check on validation set
relp_17_v = validation_x.apply(lambda x: group_relp_17(x), axis=1)
relp_17_v_x = validation_x[relp_17_v]
relp_17_v_y = validation_y[relp_17_v]

relp_17_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
relp_17_clf.fit(relp_17_train_x, relp_17_train_y)
relp_17_train_pred = relp_17_clf.predict(relp_17_train_x)
relp_17_val_pred = relp_17_clf.predict(relp_17_v_x)

print("Training Error")
print(metrics.zero_one_loss(relp_17_train_y, relp_17_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(relp_17_v_y, relp_17_val_pred))

# Age 57

In [ ]:
def group_age_57(x):
  if x['AGEP'] == 57:
    return True
  else:
    return False

def group_age_55_58(x):
  if x['AGEP'] >= 55 and x['AGEP'] <= 58:
    return True
  else:
    return False


# getting the right training set
fifty_seven_tf = train_x.apply(lambda x: group_age_57(x), axis=1)
fifty_seven_train_y = train_y[fifty_seven_tf]
fifty_seven_train_x = train_x[fifty_seven_tf]

# getting the right training set
range_fifty_seven_tf = train_x.apply(lambda x: group_age_55_58(x), axis=1)
range_fifty_seven_train_y = train_y[range_fifty_seven_tf]
range_fifty_seven_train_x = train_x[range_fifty_seven_tf]

# check on validation set
fifty_seven_v = validation_x.apply(lambda x: group_age_57(x), axis=1)
fifty_seven_v_x = validation_x[fifty_seven_v]
fifty_seven_v_y = validation_y[fifty_seven_v]

fifty_seven_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
fifty_seven_clf.fit(range_fifty_seven_train_x, range_fifty_seven_train_y)
fifty_seven_train_pred = fifty_seven_clf.predict(fifty_seven_train_x)
fifty_seven_val_pred = fifty_seven_clf.predict(fifty_seven_v_x)

print("Training Error")
print(metrics.zero_one_loss(fifty_seven_train_y, fifty_seven_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(fifty_seven_v_y, fifty_seven_val_pred))

# Age 25

In [ ]:
def group_age_25(x):
  if x['AGEP'] == 25:
    return True
  else:
    return False

def group_age_24_26(x):
  if x['AGEP'] >= 24 and x['AGEP'] <= 26:
    return True
  else:
    return False


# getting the right training set
twenty_five_tf = train_x.apply(lambda x: group_age_25(x), axis=1)
twenty_five_train_y = train_y[twenty_five_tf]
twenty_five_train_x = train_x[twenty_five_tf]

# getting the right training set
range_twenty_five_tf = train_x.apply(lambda x: group_age_24_26(x), axis=1)
range_twenty_five_train_y = train_y[range_twenty_five_tf]
range_twenty_five_train_x = train_x[range_twenty_five_tf]

# check on validation set
twenty_five_v = validation_x.apply(lambda x: group_age_25(x), axis=1)
twenty_five_v_x = validation_x[twenty_five_v]
twenty_five_v_y = validation_y[twenty_five_v]

twenty_five_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
twenty_five_clf.fit(twenty_five_train_x, twenty_five_train_y)
twenty_five_train_pred = twenty_five_clf.predict(twenty_five_train_x)
twenty_five_val_pred = twenty_five_clf.predict(twenty_five_v_x)

print("Training Error")
print(metrics.zero_one_loss(twenty_five_train_y, twenty_five_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(twenty_five_v_y, twenty_five_val_pred))

# Age 69

In [ ]:
def group_age_69(x):
  if x['AGEP'] == 69:
    return True
  else:
    return False

def group_age_68_71(x):
  if x['AGEP'] >= 68 and x['AGEP'] <= 71:
    return True
  else:
    return False


# getting the right training set
sixty_nine_tf = train_x.apply(lambda x: group_age_69(x), axis=1)
sixty_nine_train_y = train_y[sixty_nine_tf]
sixty_nine_train_x = train_x[sixty_nine_tf]

# getting the right training set
range_sixty_nine_tf = train_x.apply(lambda x: group_age_68_71(x), axis=1)
range_sixty_nine_train_y = train_y[range_sixty_nine_tf]
range_sixty_nine_train_x = train_x[range_sixty_nine_tf]

# check on validation set
sixty_nine_v = validation_x.apply(lambda x: group_age_69(x), axis=1)
sixty_nine_v_x = validation_x[sixty_nine_v]
sixty_nine_v_y = validation_y[sixty_nine_v]

sixty_nine_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
sixty_nine_clf.fit(range_sixty_nine_train_x, range_sixty_nine_train_y)
sixty_nine_train_pred = sixty_nine_clf.predict(sixty_nine_train_x)
sixty_nine_val_pred = sixty_nine_clf.predict(sixty_nine_v_x)

print("Training Error")
print(metrics.zero_one_loss(sixty_nine_train_y, sixty_nine_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sixty_nine_v_y, sixty_nine_val_pred))

# Age 58

In [ ]:
def group_age_58(x):
  if x['AGEP'] == 58:
    return True
  else:
    return False

def group_age_57_59(x):
  if x['AGEP'] >= 57 and x['AGEP'] <= 59:
    return True
  else:
    return False


# getting the right training set
fifty_eight_tf = train_x.apply(lambda x: group_age_58(x), axis=1)
fifty_eight_train_y = train_y[fifty_eight_tf]
fifty_eight_train_x = train_x[fifty_eight_tf]

# getting the right training set
range_fifty_eight_tf = train_x.apply(lambda x: group_age_57_59(x), axis=1)
range_fifty_eight_train_y = train_y[range_fifty_eight_tf]
range_fifty_eight_train_x = train_x[range_fifty_eight_tf]

# check on validation set
fifty_eight_v = validation_x.apply(lambda x: group_age_58(x), axis=1)
fifty_eight_v_x = validation_x[fifty_eight_v]
fifty_eight_v_y = validation_y[fifty_eight_v]

fifty_eight_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
fifty_eight_clf.fit(fifty_eight_train_x, fifty_eight_train_y)
fifty_eight_train_pred = fifty_eight_clf.predict(fifty_eight_train_x)
fifty_eight_val_pred = fifty_eight_clf.predict(fifty_eight_v_x)

print("Training Error")
print(metrics.zero_one_loss(fifty_eight_train_y, fifty_eight_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(fifty_eight_v_y, fifty_eight_val_pred))



# Age 59

In [ ]:
def group_age_59(x):
  if x['AGEP'] == 59:
    return True
  else:
    return False

def group_age_58_60(x):
  if x['AGEP'] >= 58 and x['AGEP'] <= 60:
    return True
  else:
    return False


# getting the right training set
fifty_nine_tf = train_x.apply(lambda x: group_age_59(x), axis=1)
fifty_nine_train_y = train_y[fifty_nine_tf]
fifty_nine_train_x = train_x[fifty_nine_tf]

# getting the right training set
range_fifty_nine_tf = train_x.apply(lambda x: group_age_58_60(x), axis=1)
range_fifty_nine_train_y = train_y[range_fifty_nine_tf]
range_fifty_nine_train_x = train_x[range_fifty_nine_tf]

# check on validation set
fifty_nine_v = validation_x.apply(lambda x: group_age_59(x), axis=1)
fifty_nine_v_x = validation_x[fifty_nine_v]
fifty_nine_v_y = validation_y[fifty_nine_v]

fifty_nine_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=5, n_estimators=500)
fifty_nine_clf.fit(range_fifty_nine_train_x, range_fifty_nine_train_y)
fifty_nine_train_pred = fifty_nine_clf.predict(fifty_nine_train_x)
fifty_nine_val_pred = fifty_nine_clf.predict(fifty_nine_v_x)

print("Training Error")
print(metrics.zero_one_loss(fifty_nine_train_y, fifty_nine_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(fifty_nine_v_y, fifty_nine_val_pred))



# Age 24

In [ ]:
def group_age_24(x):
  if x['AGEP'] == 24:
    return True
  else:
    return False

def group_age_23_25(x):
  if x['AGEP'] >= 23 and x['AGEP'] <= 25:
    return True
  else:
    return False

# getting the right training set
twenty_four_tf = train_x.apply(lambda x: group_age_24(x), axis=1)
twenty_four_train_y = train_y[twenty_four_tf]
twenty_four_train_x = train_x[twenty_four_tf]

# getting the right training set
range_twenty_four_tf = train_x.apply(lambda x: group_age_23_25(x), axis=1)
range_twenty_four_train_y = train_y[range_twenty_four_tf]
range_twenty_four_train_x = train_x[range_twenty_four_tf]

# check on validation set
twenty_four_v = validation_x.apply(lambda x: group_age_24(x), axis=1)
twenty_four_v_x = validation_x[twenty_four_v]
twenty_four_v_y = validation_y[twenty_four_v]

twenty_four_clf = RandomForestClassifier(bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=3, min_samples_split=6, n_estimators=500)
twenty_four_clf.fit(range_twenty_four_train_x, range_twenty_four_train_y)
twenty_four_train_pred = twenty_four_clf.predict(twenty_four_train_x)
twenty_four_val_pred = twenty_four_clf.predict(twenty_four_v_x)

print("Training Error")
print(metrics.zero_one_loss(twenty_four_train_y, twenty_four_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(twenty_four_v_y, twenty_four_val_pred))



# Age 68

In [ ]:
def group_age_68(x):
  if x['AGEP'] == 68:
    return True
  else:
    return False

def group_age_66_70(x):
  if x['AGEP'] >= 66 and x['AGEP'] <= 70:
    return True
  else:
    return False

# getting the right training set
sixty_eight_tf = train_x.apply(lambda x: group_age_68(x), axis=1)
sixty_eight_train_y = train_y[sixty_eight_tf]
sixty_eight_train_x = train_x[sixty_eight_tf]

# getting the right training set
range_sixty_eight_tf = train_x.apply(lambda x: group_age_66_70(x), axis=1)
range_sixty_eight_train_y = train_y[range_sixty_eight_tf]
range_sixty_eight_train_x = train_x[range_sixty_eight_tf]

# check on validation set
sixty_eight_v = validation_x.apply(lambda x: group_age_68(x), axis=1)
sixty_eight_v_x = validation_x[sixty_eight_v]
sixty_eight_v_y = validation_y[sixty_eight_v]

sixty_eight_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=6, n_estimators=500)
sixty_eight_clf.fit(sixty_eight_train_x, sixty_eight_train_y)
sixty_eight_train_pred = sixty_eight_clf.predict(sixty_eight_train_x)
sixty_eight_val_pred = sixty_eight_clf.predict(sixty_eight_v_x)

print("Training Error")
print(metrics.zero_one_loss(sixty_eight_train_y, sixty_eight_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(sixty_eight_v_y, sixty_eight_val_pred))



# Age 18

In [ ]:
def group_age_18(x):
  if x['AGEP'] == 18:
    return True
  else:
    return False

def group_age_17_20(x):
  if x['AGEP'] >= 17 and x['AGEP'] <= 20:
    return True
  else:
    return False

# getting the right training set
eighteen_tf = train_x.apply(lambda x: group_age_18(x), axis=1)
eighteen_train_y = train_y[eighteen_tf]
eighteen_train_x = train_x[eighteen_tf]

# getting the right training set
range_eighteen_tf = train_x.apply(lambda x: group_age_22_25(x), axis=1)
range_eighteen_train_y = train_y[range_eighteen_tf]
range_eighteen_train_x = train_x[range_eighteen_tf]

# check on validation set
eighteen_v = validation_x.apply(lambda x: group_age_18(x), axis=1)
eighteen_v_x = validation_x[eighteen_v]
eighteen_v_y = validation_y[eighteen_v]

eighteen_clf = RandomForestClassifier(bootstrap=True, max_depth=9, max_features=3, min_samples_leaf=3, min_samples_split=6, n_estimators=500)
eighteen_clf.fit(eighteen_train_x, eighteen_train_y)
eighteen_train_pred = eighteen_clf.predict(eighteen_train_x)
eighteen_val_pred = eighteen_clf.predict(eighteen_v_x)

print("Training Error")
print(metrics.zero_one_loss(eighteen_train_y, eighteen_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(eighteen_v_y, eighteen_val_pred))



Seems as if KNN did not work as well. We should update the PDL with the random forest model!

In [ ]:
updater(group_age_retirement, retirement_clf.predict, group_name="retirement age")

# SCHL=16 (HS diploma)

In [ ]:
# define the new group for the classifier
def group_hs_more(x):
  if x['SCHL'] == 16 or x['SCHL'] == 15 or x['SCHL'] == 17 :
    return True
  else:
    return False

def group_hs(x):
  if x['SCHL'] == 16 or x['SCHL'] == 15 or x['SCHL'] == 17 :
    return True
  else:
    return False

# get the correct training set first
more_hs = train_x.apply(lambda x: group_hs_more(x), axis=1)
more_hs_train_y = train_y[more_hs]
more_hs_train_x = train_x[more_hs]

# get the correct training set first
hs = train_x.apply(lambda x: group_hs(x), axis=1)
hs_train_y = train_y[hs]
hs_train_x = train_x[hs]

# check on validation set
hs_v = validation_x.apply(lambda x: group_hs(x), axis=1)
hs_v_x = validation_x[hs_v]
hs_v_y = validation_y[hs_v]

hs_clf = RandomForestClassifier(bootstrap=True, max_depth=8, max_features=3, min_samples_leaf=3, min_samples_split=6, n_estimators=500)
hs_clf.fit(more_hs_train_x, more_hs_train_y)
hs_train_pred = hs_clf.predict(hs_train_x)
hs_val_pred = hs_clf.predict(hs_v_x)

print("Training Error")
print(metrics.zero_one_loss(hs_train_y, hs_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(hs_v_y, hs_val_pred))

# Limited College Education (SCHL=19)

Another underperforming group is SCHL=19, which are people with limited college (1 or more years of college credit but no degree). Logically, there are dropouts who end up employed and well-off (Steve Jobs?) and those who don't. That's why this group is so hard to predict.

In [ ]:
# define the new group for the classifier
def group_limited_college(x):
  if x['SCHL'] == 19:
    return True
  else:
    return False

# get the correct training set first
limited_college = train_x.apply(lambda x: group_limited_college(x), axis=1)
limited_college_train_y = train_y[limited_college]
limited_college_train_x = train_x[limited_college]

# check on validation set
limited_college_v = validation_x.apply(lambda x: group_limited_college(x), axis=1)
limited_college_v_x = validation_x[limited_college_v]
limited_college_v_y = validation_y[limited_college_v]

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(7, 14, num = 8)]
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [1, 2, 4, 6]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [ ]:
limited_college_clf = RandomForestClassifier()
limited_college_clf_rand = RandomizedSearchCV(estimator = limited_college_clf, param_distributions = random_grid, n_iter = 70, cv = 3, verbose=2, random_state=42, n_jobs = -1)
limited_college_clf_rand.fit(limited_college_train_x, limited_college_train_y)
print(limited_college_clf_rand.best_params_)

In [ ]:
param_grid_limited_college = {
    'bootstrap': [True],
    'max_depth': [11, 12],
    'max_features': [2, 3],
    'min_samples_leaf': [1, 3],
    'min_samples_split': [5, 7],
    'n_estimators': [400, 500]
}

grid_search_limited_college = GridSearchCV(estimator = limited_college_clf, param_grid = param_grid_limited_college, cv = 3, n_jobs = -1, verbose = 2)
grid_search_limited_college.fit(limited_college_train_x, limited_college_train_y)
print(grid_search_limited_college.best_params_)

In [ ]:
limited_college_clf = RandomForestClassifier(bootstrap=True, max_depth=12, max_features=3, min_samples_leaf=1, min_samples_split=7, n_estimators=500)
limited_college_clf.fit(limited_college_train_x, limited_college_train_y)
limited_college_train_pred = limited_college_clf.predict(limited_college_train_x)
limited_college_val_pred = limited_college_clf.predict(limited_college_v_x)

print("Training Error")
print(metrics.zero_one_loss(limited_college_train_y, limited_college_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(limited_college_v_y, limited_college_val_pred))


# Non-Citizens

The next group that seems to be underperforming is CIT=5.0, which are non-US citizens. 

In [ ]:
def group_noncit(x):
  if x['CIT'] == 5:
    return True
  else:
    return False

# get the correct training set first
noncit = train_x.apply(lambda x: group_noncit(x), axis=1)
noncit_train_x = train_x[noncit]
noncit_train_y = train_y[noncit]

# get the correct validation data
noncit_v = validation_x.apply(lambda x: group_noncit(x), axis=1)
noncit_v_x = validation_x[noncit_v]
nonvit_v_y = validation_y[noncit_v]

In [ ]:
noncit_clf = RandomForestClassifier()
noncit_clf_rand = RandomizedSearchCV(estimator = noncit_clf, param_distributions = random_grid, n_iter = 70, cv = 3, verbose=2, random_state=42, n_jobs = -1)
noncit_clf_rand.fit(noncit_train_x, noncit_train_y)
print(noncit_clf_rand.best_params_)

In [ ]:
param_grid_noncit = {
    'bootstrap': [True],
    'max_depth': [14, 15],
    'max_features': [2, 3],
    'min_samples_leaf': [4, 5],
    'min_samples_split': [5, 7],
    'n_estimators': [200, 300]
}

grid_search_noncit = GridSearchCV(estimator = noncit_clf, param_grid = param_grid_noncit, cv = 3, n_jobs = -1, verbose = 2)
grid_search_noncit.fit(noncit_train_x, noncit_train_y)
print(grid_search_noncit.best_params_)

In [ ]:
noncit_clf = RandomForestClassifier(bootstrap=True, max_depth=10, max_features=3, min_samples_leaf=4, min_samples_split=7, n_estimators=300)
noncit_clf.fit(noncit_train_x, noncit_train_y)
noncit_train_pred = noncit_clf.predict(noncit_train_x)
noncit_val_pred = noncit_clf.predict(noncit_v_x)

print("Training Error")
print(metrics.zero_one_loss(noncit_train_y, noncit_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(noncit_v_y, noncit_val_pred))


We are going to try a KNN approach next. They all seem pretty bad compared to the Random Forest Model.

In [ ]:
# try a k nearest neighbors approach as well
print("K Nearest Neighbors")
neighbors = [3, 4, 5, 6, 7, 8]

for n in neighbors:
  print("Neighbors:", n)
  noncit_knn = KNeighborsClassifier(n_neighbors=n)
  noncit_knn.fit(noncit_train_x, noncit_train_y)
  noncit_train_knn_pred = noncit_knn.predict(noncit_train_x)
  print("Training Set Error")
  print(metrics.zero_one_loss(noncit_train_y, noncit_train_knn_pred))

  noncit_v_knn_pred = noncit_knn.predict(noncit_v_x)
  print("Validation Set Error")
  print(metrics.zero_one_loss(noncit_v_y, noncit_v_knn_pred))

# 12th Graders Without Diplomas (SCHL=15)

In [ ]:
def group_twelve(x):
  if x['SCHL'] == 15:
    return True
  else:
    return False

# get the correct training set first
twelve = train_x.apply(lambda x: group_twelve(x), axis=1)
twelve_train_x = train_x[twelve]
twelve_train_y = train_y[twelve]

# get the correct validation data
twelve_v = validation_x.apply(lambda x: group_twelve(x), axis=1)
twelve_v_x = validation_x[twelve_v]
twelve_v_y = validation_y[twelve_v]

In [ ]:
twelve_clf = RandomForestClassifier()
twelve_clf_rand = RandomizedSearchCV(estimator = twelve_clf, param_distributions = random_grid, n_iter = 70, cv = 3, verbose=2, random_state=42, n_jobs = -1)
twelve_clf_rand.fit(twelve_train_x, twelve_train_y)
print(twelve_clf_rand.best_params_)

In [ ]:
param_grid_twelve = {
    'bootstrap': [True],
    'max_depth': [8, 9, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [5, 6, 7],
    'min_samples_split': [2, 4],
    'n_estimators': [500, 700]
}

grid_search_twelve = GridSearchCV(estimator = twelve_clf, param_grid = param_grid_twelve, cv = 3, n_jobs = -1, verbose = 2)
grid_search_twelve.fit(twelve_train_x, twelve_train_y)
print(grid_search_twelve.best_params_)

In [ ]:
twelve_clf = RandomForestClassifier(bootstrap=True, max_depth=10, max_features=4, min_samples_leaf=6, min_samples_split=3, n_estimators=500)
twelve_clf.fit(twelve_train_x, twelve_train_y)
twelve_train_pred = twelve_clf.predict(twelve_train_x)
twelve_val_pred = twelve_clf.predict(twelve_v_x)

print("Training Error")
print(metrics.zero_one_loss(twelve_train_y, twelve_train_pred))
print("Validation Error")
print(metrics.zero_one_loss(twelve_v_y, twelve_val_pred))

In [ ]:
def group_noncit(x):
  if x['CIT'] == 5:
    return True
  else:
    return False

# get the correct training set first
noncit = train_x.apply(lambda x: group_noncit(x), axis=1)
noncit_train_x = train_x[noncit]
noncit_train_y = train_y[noncit]

depths = [8, 9, 10, 11, 12, 13, 14, 15, 16]
best_val = 1
best_depth = 0

print("Random Forest Classifier:")
for d in depths:
  # build a random forest classifier for it 
  print("Depth:", d)
  noncit_clf = RandomForestClassifier(n_estimators=100, max_features=8, max_depth=d)
  noncit_clf.fit(noncit_train_x, noncit_train_y)
  noncit_train_pred = noncit_clf.predict(noncit_train_x)
  print("Training Set Error")
  print(metrics.zero_one_loss(noncit_train_y, noncit_train_pred))

  # check on validation set
  noncit_v = validation_x.apply(lambda x: group_noncit(x), axis=1)
  noncit_v_x = validation_x[noncit_v]
  noncit_v_y = validation_y[noncit_v]

  noncit_v_pred = noncit_clf.predict(noncit_v_x)
  print("Validation Set Error")
  print(metrics.zero_one_loss(noncit_v_y, noncit_v_pred))
  if metrics.zero_one_loss(noncit_v_y, noncit_v_pred) < best_val:
    best_val = metrics.zero_one_loss(noncit_v_y, noncit_v_pred)
    best_depth = d

print("Best Validation Score:", best_val, "in depth", best_depth)

Indeed splitting up into education category results in better models! We're also going to build a random forest classifier for educated noncitizens.

In [ ]:
def group_edu_noncit(x):
  if x['CIT'] == 5 and x['SCHL'] > 19:
    return True
  else:
    return False

# get the correct training set first
edu_noncit = train_x.apply(lambda x: group_edu_noncit(x), axis=1)
edu_noncit_train_x = train_x[edu_noncit]
edu_noncit_train_y = train_y[edu_noncit]

# check on validation set
edu_noncit_v = validation_x.apply(lambda x: group_edu_noncit(x), axis=1)
edu_noncit_v_x = validation_x[edu_noncit_v]
edu_noncit_v_y = validation_y[edu_noncit_v]

depths = [8, 9, 10, 11, 12, 13, 14, 15, 16]
best_val = 1
best_depth = 0

print("Random Forest Classifier:")
for d in depths:
  # build a random forest classifier for it 
  print("Depth:", d)
  edu_noncit_clf = RandomForestClassifier(n_estimators=60, max_features=8, max_depth=d)
  edu_noncit_clf.fit(edu_noncit_train_x, edu_noncit_train_y)
  edu_noncit_train_pred = edu_noncit_clf.predict(edu_noncit_train_x)
  print("Training Set Error")
  print(metrics.zero_one_loss(edu_noncit_train_y, edu_noncit_train_pred))

  edu_noncit_v_pred = edu_noncit_clf.predict(edu_noncit_v_x)
  print("Validation Set Error")
  print(metrics.zero_one_loss(edu_noncit_v_y, edu_noncit_v_pred))
  if metrics.zero_one_loss(edu_noncit_v_y, edu_noncit_v_pred) < best_val:
    best_val = metrics.zero_one_loss(edu_noncit_v_y, edu_noncit_v_pred)
    best_depth = d

  print("Best depth is:", best_depth, "with error", best_val)

Best depth is 10! That's our classifier for the educated noncitizens.

In [ ]:
edu_noncit_clf = RandomForestClassifier(n_estimators=60, max_features=8, max_depth=10)
edu_noncit_clf.fit(edu_noncit_train_x, edu_noncit_train_y)
edu_noncit_train_pred = edu_noncit_clf.predict(edu_noncit_train_x)
print("Training Set Error")
print(metrics.zero_one_loss(edu_noncit_train_y, edu_noncit_train_pred))

In [ ]:
def group_edu_noncit_updater(x):
  if x['CIT'] == 5 and x['SCHL'] > 19:
    return 1
  else:
    return 0

def group_unedu_noncit_updater(x):
  if x['CIT'] == 5 and x['SCHL'] <= 19:
    return 1
  else:
    return 0

In [ ]:
updater(group_edu_noncit_updater, edu_noncit_clf.predict, group_name="Educated Noncitizens")

In [ ]:
updater(group_unedu_noncit_updater, unedu_noncit_clf.predict, group_name="No College Degree Noncitizens")

Another subgroup that performs poorly is SCHL=19, which are people with one year of college (but have not finished). Again, this makes logical sense as a subgroup that is difficult to predict.

Turns out the model for noncitizens with no college degree performs worse! 

Once you've found a promising group g, you can run the following updater code. Here, we define two different update functions. The first, `simple_updater`, only requires that you find some model g that you think f might do poorly on. Then, it automatically trains a decision list of depth 10 on the training data restricted to your g, and it passes that model and g along to the updater.

You might want to do something a bit fancier than a decision tree to make your model, in which case you can run the second updater, which takes as input a group g and model h, and then updates f accordingly.

Every time you run the update function, it will tell you if your (g,h) passed the validation checks, i.e. if a) your group existed in the validation data and b) it made an improvement compared to f. If it did pass the validation checks, then the model f is updated to include your g and h. **Note that this means that as you run updates, it will be increasingly difficult to find groups that make improvements.**

In [ ]:
def simple_updater(g,group_name = "g"):
    # if you want to change how h is trained, you can edit the below line.
    h = bountyHuntWrapper.build_model(train_x, train_y, g, dt_depth=10)
    # do not change anything beyond this point.
    if bountyHuntWrapper.run_checks(f, validation_x, validation_y, g, h, train_x=train_x, train_y=train_y):
        print("Running Update")
        bountyHuntWrapper.run_updates(f, g, h, train_x, train_y, validation_x, validation_y, group_name=group_name)

def updater(g, h, group_name="g"):
    # do not alter this code
    if bountyHuntWrapper.run_checks(f, validation_x, validation_y, g, h, train_x=train_x, train_y=train_y):
        print("Running Update")
        bountyHuntWrapper.run_updates(f, g, h, train_x, train_y, validation_x, validation_y, group_name=group_name)

In the below block, provide a script that builds *the entire final PDL that you come up with*. We will run this on the initial version of f (the decision stump) in order to evaluate your code. (Note: it is fine for the group functions g and the hs to be defined in above code blocks, just make sure that everything runs as you expect if you run everything from a clean kernel)

In [ ]:
def group_female(x):
  if x['SEX'] == 2:
    return 1
  else:
    return 0

def group_nonmilitary(x):
  if x['MIL'] == 4:
    return 1
  else:
    return 0

def group_foreigners(x):
  if x['NATIVITY'] == 2:
    return 1
  else:
    return 0

def group_naturalized(x):
  if x['CIT'] == 4:
    return 1
  else:
    return 0

def group_divorced(x):
  if x['MAR'] == 3:
    return 1
  else:
    return 0

def group_asian(x):
  if x['RAC1P'] == 6:
    return 1
  else:
    return 0

def group_lim_college(x):
  if x['SCHL'] == 19:
    return 1
  else:
    return 0

def group_age_64(x):
  if x['AGEP'] == 64:
    return 1
  else:
    return 0
  
def group_age_20(x):
  if x['AGEP'] == 20:
    return 1
  else:
    return 0

def group_age_25(x):
  if x['AGEP'] == 25:
    return 1
  else:
    return 0

In [ ]:
updater(group_age_64, sixty_four_clf.predict, group_name="age 64")


In [ ]:
# updater(group_female, sex_2_clf.predict, group_name="females")
# updater(group_nonmilitary, mil_4_clf.predict, group_name="nonmilitary")
# updater(group_foreigners, nativity_2_clf.predict, group_name="foreigners")
# updater(group_naturalized, cit_4_clf.predict, group_name="naturalized citizens")
# updater(group_divorced, mar_3_clf.predict, group_name="divorced")
# updater(group_asian, rac1p_6_clf.predict, group_name="asians")
simple_updater(group_age_20, group_name="age 20")


In [ ]:
updater(group_lim_college, limited_college_clf.predict, group_name="limited college")

In [ ]:
updater(group_age_25, twenty_five_clf.predict, group_name="age 25")

# Saving Your Model

We'd like to output the PDL to some permanent location for grading purposes. The lines below do this.

In [ ]:
import dill as pickle # you will probably need to install dill, which you do w pip install dill in your command line
with open('pdl.pkl', 'wb') as pickle_file:
    pickle.dump(f, pickle_file)

If you saved your PDL to pdl.pkl and you want to reload it, you can do so as follows (instead of re-building it from scratch every time you shut down you kernel). Just be sure that your final PDL is fully replicable in the final version of your code, so that we can re-build it just given your gs and hs.

In [ ]:
with open('pdl.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)

# Analysis of Your Final Model

1. How does your final model perform? On both the validation set and the training data, calculate f's error rates on each of the groups you identified, calculate the error rates of the initial model on each of the groups you identified, and compare them by taking their difference. Hint: you can use the helper function `bountyHuntWrapper.measure_group_error(model, g, x, y)` to get the error of f on x and y restricted to just the datapoints in a group g, and you can use `metrics.zero_one_loss` for the initial model (which is just a DL so you can directly use the scikit.learn functions on it).
2. Say instead you used bootstrapped fairness to postprocess equal error rates on the initial model over the groups you discovered (assuming you had a way to identify those groups ahead of time). How much would you need to inflate each groups' error to get them equal?

# Final Model Analysis

1. On the training data, the initial model's errors are $[0.17245931126519476, 0.3322757111597374, 0.33186920448999513, 0.2704069347459668, 0.2134478424801005, 0.23570019723865876, 0.37905759162303665, 0.36776119402985075, 0.4032712915961647, 0.3622244488977956]$ while the final model's errors are $[0.1623125126925643, 0.2673960612691466, 0.26403123474865786, 0.24271610883698536, 0.15919564306661083, 0.2238658777120316, 0.268586387434555, 0.2764179104477612, 0.2831359278059785, 0.2865731462925851]$. 

On the validation set, the initial model's errors are $[0.17410729395836855, 0.36441532258064513, 0.32552693208430916, 0.2751042061386889, 0.22736418511066403, 0.2733990147783252, 0.375, 0.4222222222222223, 0.41647058823529415, 0.40452261306532666]$, while the final model's errors are $[0.17173802673887295, 0.3407258064516129, 0.32084309133489464, 0.2743463433118606, 0.1951710261569416, 0.2660098522167488, 0.35465116279069764, 0.37777777777777777, 0.36, 0.3919597989949749]$.

Therefore, the improvement on error for the training set is $[0.010146798572630455, 0.06487964989059081, 0.06783796974133727, 0.02769082590898142, 0.05425219941348969, 0.011834319526627168, 0.11047120418848166, 0.09134328358208954, 0.12013536379018619, 0.07565130260521047]$.

The improvement on the validation set is $[0.0023692672194955966, 0.02368951612903225, 0.004683840749414525, 7.578628268283261E-4, 0.032193158953722434, 0.007389162561576401, 0.02034883720930236, 0.04444444444444451, 0.05647058823529416, 0.012562814070351758]$.

2. The final group errors on all groups are $[0.17173802673887295, 0.3407258064516129, 0.32084309133489464, 0.2743463433118606, 0.1951710261569416, 0.2660098522167488, 0.35465116279069764, 0.37777777777777777, 0.36, 0.3919597989949749]$. 

Therefore, if we want to equalize the errors, we would need $Max(E) - E_i$, and therefore the array to inflate each error is $[0.2202217723, 0.05123399254, 0.07111670766, 0.1176134557, 0.1967887728 , 0.2660098522167488, 0.0373086362, 0.01418202122, 0.03195979899, 0]$.

# Data Exploration

In order to find promising groups, you may find it helpful to do some data exploration. Please include any code or visualizations that you did to do so here. To get you started, here are some things you may find useful:

1. How to grab the predictions of the current PDL on the training data: because f.predict takes a single value as input, you have to use an apply function for this.

In [ ]:
preds = train_x.apply(f.predict, axis=1)

2. Getting the zero-one loss of a model restricted to a group you have defined.

In [ ]:
g = lambda x: 1 #here we define a group that just is all the data, replace as you see fit.
print("Total Error")
print(bountyHuntWrapper.measure_group_error(f, g, train_x, train_y))

The below function determines how well the model performs for certain subgroups

In [ ]:
df = pd.DataFrame(columns=['Col Name', 'Value', 'Error', 'Size'])

cols = list(train_x.columns)
for col in cols:
  vals = train_x[col].unique()
  for val in vals:
    pred_group = preds[train_x[col] == val]
    if (len(pred_group) >= 1000):
      actual = train_y[train_x[col] == val]
      group_errors = metrics.zero_one_loss(actual, pred_group)
      df2 = {'Col Name': col, 'Value': val, 'Error': group_errors, 'Size': len(pred_group)}
      df = df.append(df2, ignore_index = True)

df = df.sort_values(by='Error', axis=0, ascending=False)
print(df.to_string())

3. You can view the training data by calling `train_x`. If you want to only view the data for a single group defined by your group function, you can run the following:

In [ ]:
# replace g with whatever your group is
indices = train_x.apply(g, axis=1) == 1
xs = train_x[indices]
ys = train_y[indices]

4. Inspecting the existing PDL: The PDL is stored as an object, and tracks its training errors, validation set errors, and the group functions that are used in lists where the ith element is the group errors of all groups discovered so far on the ith node in the PDL. If you are more curious about the implementation, you can look at the model.py file in the codebase, which doesn't contain anything you can use to adaptively modify your code. (But lives in the same folder as the rest of the codebase just to make importing things easier)

In [ ]:
# f is the current model
print(f.train_errors) # group errors on training set.
print(f.train_errors[0]) # this is the group error of each group on the initial PDL. The ith element of f.train_errors is the group error of each group on the ith version of the PDL.
print(f.test_errors) # group errors on validation set
print(f.predicates) # all of the group functions that have been appended so far
print(f.leaves) # all of the h functions appended so far
print(f.pred_names) # the names you passed in for each of the group functions, to more easily understand which are which.

5. Looking at the group error of the ith group over each round of updates: Say you found a group at round 5 and you want to know how its group error looked at previous or subsequent rounds. To do so, you can pull `f.train_errors` or `f.test_errors` and look at the ith element of each list as follows:

In [ ]:
target_group = 0 # this sets the group whose error you want to look at at each round to the initial model. If I wanted to look at the 1st group introduced, would change to a 1, e.g.
group_errs = [f.train_errors[i][target_group] for i in range(len(f.train_errors))]
group_errs